In [4]:
import pandas as pd

# 加载你的原始数据
df = pd.read_excel('./originial_data_0331.xlsx')

# 重命名列名，去除特殊符号和空格，便于 Tableau 使用
df.rename(columns={
    'Who contacts the other party (Customer, Company, None)': 'Contact_Party',
    'External (customer contact) vs. internal (data center, agent support etc) ': 'Contact_Type'
}, inplace=True)

# 定义要处理的类别列
category_cols = ['Cat 1', 'Cat 2', 'Cat 3', 'Cat 4']

# 数据清理函数（清除特殊字符和空格）
for col in category_cols:
    df[col] = df[col].replace({'`':'', '-':'', 'None':'', ' ':'', 'nan':''}, regex=True)

# 将多选类别数据展开成多行
df_expanded = df.copy()
for col in category_cols:
    df_expanded[col] = df_expanded[col].astype(str).str.split(';')

df_expanded = df_expanded.explode('Cat 1')\
                         .explode('Cat 2')\
                         .explode('Cat 3')\
                         .explode('Cat 4')

# 去除缺失类别的行
df_expanded = df_expanded[
    (df_expanded['Cat 1'] != '') &
    (df_expanded['Cat 2'] != '') &
    (df_expanded['Cat 3'] != '') &
    (df_expanded['Cat 4'] != '')
]

# 为每个类别创建清晰标签（根据你原本给出的规则定义）
cat1_labels = {
    '1': 'Summarization & Insights',
    '2': 'Personalization & Engagement',
    '3': 'Content Creation & Enhancement',
    '4': 'Customer Support & Query Resolution',
    '5': 'Process Automation',
    '6': 'Predictive Analytics & Sentiment Analysis'
}

cat2_labels = {
    '1': 'Conversational AI',
    '2': 'Multi-Agent System',
    '3': 'Multimodal AI',
    '4': 'Process Automation (RPA)',
    '5': 'Marketing & Personalized AI',
    '6': 'Predictive Analysis',
    '7': 'Sentiment Analysis & Customer Insights'
}

cat3_labels = {
    '1': 'Awareness & Brand Discovery',
    '2': 'Consideration & Purchase Decision-Making',
    '3': 'Service Usage & Ongoing Support',
    '4': 'Customer Retention & Loyalty',
    '5': 'Exit & Feedback Optimization'
}

cat4_labels = {
    '1': 'Text-to-Text AI',
    '2': 'Text-to-Image AI',
    '3': 'Text-to-Voice AI',
    '4': 'Voice-to-Text AI',
    '5': 'Voice-to-Audio AI'
}

# 映射类别标签
df_expanded['Cat 1 Label'] = df_expanded['Cat 1'].map(cat1_labels)
df_expanded['Cat 2 Label'] = df_expanded['Cat 2'].map(cat2_labels)
df_expanded['Cat 3 Label'] = df_expanded['Cat 3'].map(cat3_labels)
df_expanded['Cat 4 Label'] = df_expanded['Cat 4'].map(cat4_labels)

# 保存预处理完成的数据为 CSV（方便 Tableau 导入）
df_expanded.to_csv('./tableau_ready_data.csv', index=False)